In [1]:
!unzip /content/drive-download-20210422T144453Z-001.zip -d  /content/data 

Archive:  /content/drive-download-20210422T144453Z-001.zip
  inflating: /content/data/test_resized/pug/pug6.png  
  inflating: /content/data/test_resized/pug/pug5.png  
  inflating: /content/data/test_resized/pug/pug7.png  
  inflating: /content/data/test_resized/pug/pug10.png  
  inflating: /content/data/test_resized/pug/pug9.png  
  inflating: /content/data/test_resized/pug/pug3.png  
  inflating: /content/data/test_resized/pug/pug4.png  
  inflating: /content/data/test_resized/pug/pug2.png  
  inflating: /content/data/test_resized/pug/pug1.png  
  inflating: /content/data/test_resized/pomeranian/pomeranian10.png  
  inflating: /content/data/test_resized/pomeranian/pomeranian9.png  
  inflating: /content/data/test_resized/pomeranian/pomeranian8.png  
  inflating: /content/data/test_resized/pomeranian/pomeranian7.png  
  inflating: /content/data/test_resized/pug/pug8.png  
  inflating: /content/data/test_resized/pomeranian/pomeranian3.png  
  inflating: /content/data/test_resized/pome

In [2]:
%cd /content/data

/content/data


In [7]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


from matplotlib import pyplot
from matplotlib.image import imread

def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator()
	# specify imagenet mean values for centering
	# datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('./train_resized',
		class_mode='binary', batch_size=16, target_size=(224, 224))
	test_it = datagen.flow_from_directory('./test_resized',
		class_mode='binary', batch_size=16, target_size=(224, 224))
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it), epochs=4, verbose=1)
	# evaluate model
	_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	model.save('./final_model.h5')
	# learning curves
	summarize_diagnostics(history)
 
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	# pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	# pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig("without_augment" + '_plot.png')
	pyplot.close()
 
# entry point, run the test harness
run_test_harness()

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/4
4/4 [==============================] - 20s 5s/step - loss: 24.5103 - accuracy: 0.5438
Epoch 2/4
4/4 [==============================] - 19s 5s/step - loss: 5.1385 - accuracy: 0.8929
Epoch 3/4
4/4 [==============================] - 19s 5s/step - loss: 0.0611 - accuracy: 0.9933
Epoch 4/4
4/4 [==============================] - 19s 5s/step - loss: 0.5494 - accuracy: 0.9888
> 100.000


In [5]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

 
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	# pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	# pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig("With_augment" + '_plot.png')
	pyplot.close()
 

def run_test_harness1():
  from matplotlib import pyplot
  from matplotlib.image import imread

	# define model
  model = define_model()
	# create data generator
  datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
	# specify imagenet mean values for centering
  datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
  train_it = datagen.flow_from_directory('./train_resized',
		class_mode='binary', batch_size=16, target_size=(224, 224))
  test_it = datagen.flow_from_directory('./test_resized',
		class_mode='binary', batch_size=16, target_size=(224, 224))
	# fit model
  history = model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=4, verbose=1)
	# evaluate model
  _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
  print('> %.3f' % (acc * 100.0))
	# learning curves
  summarize_diagnostics(history)
 
# entry point, run the test harness

  
  model.save('./final_model_new.h5')
 
# entry point, run the test harness
run_test_harness1()
 


Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/4
4/4 [==============================] - 21s 5s/step - loss: 2.6308 - accuracy: 0.7245
Epoch 2/4
4/4 [==============================] - 20s 5s/step - loss: 0.7017 - accuracy: 0.9177
Epoch 3/4
4/4 [==============================] - 20s 5s/step - loss: 2.6070 - accuracy: 0.9233
Epoch 4/4
4/4 [==============================] - 20s 5s/step - loss: 0.4391 - accuracy: 0.9867


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


> 95.000


In [6]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('./test_resized/pomeranian/pomeranian5.png')
	model = load_model('final_model_new.h5')
	# predict the class
	result = model.predict(img)
	if(result>0.5):
		print("Pug")
	else:
		print("Pomeranian")
	
 
# entry point, run the example
run_example()

Pomeranian
